In [1]:
%run input/Format.ipynb
!rm output/dynamic_delta_fits/*
import ROOT as root
from array import array
import math as math
root.gErrorIgnoreLevel = root.kFatal
import multiprocessing as mp
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 0
is_sigmalised = 0
sim = 0
Min_Entries = 10000
N_files = 1
Do_save = 1

In [3]:
isigma = 2
N_layers = 4
N_hists = 4
layers = ["1-2 layer east pos","1-2 layer west pos","1-2 layer east neg","1-2 layer west neg","2-3 layer east pos","2-3 layer west pos",\
          "2-3 layer east neg","2-3 layer west neg","2-4 layer east pos","2-4 layer west pos","2-4 layer east neg","2-4 layer west neg",\
          "3-4 layer east pos","3-4 layer west pos","3-4 layer east neg","3-4 layer west neg",\
          "3 layer east pos","3 layer west pos","3 layer east neg","3 layer west neg",\
          "4 layer east pos","4 layer west pos","4 layer east neg","4 layer west neg",\
          "20-40%","40-60%","60-80%","80-93%"]
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
file_path="input/"
file_names="my-m_ee_Run14AuAu_83rd_new_19842_1065runs" 
hist_select_3D_names = ["dphi_hist_el_dynamic","dthe_hist_el_dynamic","sdphi_hist_el_dynamic","sdthe_hist_el_dynamic"]
iOptames=["phi","theta"]

In [4]:
if sim :
    file_path="../sim/output/Newembed/deltas/" 
    file_path="../sim/output/Newembed/sngl/" 
    file_path="../sim/output/Newembed/deltas/250301/" 
    file_names="dalitz_50M_v3"#"bbbar_10M_v0" #piminus_pythia_1M_v0 piminus_DCA_50M_V3 photon_100M_V0 piminus_DCA_50M_V1 piminus_DCA_50M_V0_100mrad ccbar_50M_v0 piminus_DCA_50M_V0
    file_names="lepton_50M_v1"
    file_names="../../dca/piminus_50M_v0"

In [5]:
hists_read = []

infile = root.TFile.Open(file_path+file_names+f".root", "read")
for ilayer in range(N_layers):
    hist_select_3D0 = []
    for ihist in range(len(hist_select_3D_names)):
        hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{ilayer}"))
        hist_select_3D0[-1].SetDirectory(root.nullptr)
    hists_read.append(hist_select_3D0)

infile.Close()

In [6]:
z_bins = [i for i in range(9,30,5)]
z_means = array('d', [i-19.5 for i in z_bins])
z_error = array('d', [0.5 for i in z_bins])

In [7]:
def sdphi_fitter(ihits, sigma=isigma):

    iHist = hists_read[ihits][iOption0+is_sigmalised*2]

    Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, phi_local, phi_local_err = [], [], [], [], [], []

    for iz in z_bins:

        phi_z =  array( 'd' )
        phi_z_err =  array( 'd' )
        Mean_z =  array( 'd' )
        Sigma_z =  array( 'd' )
        Mean_z_err =  array( 'd' )
        Sigma_z_err =  array( 'd' )

        N_ybins_loc = iHist.GetNbinsY()
        for iy in range(N_ybins_loc):
            proj_local = iHist.ProjectionX(iHist.GetName()+f"_{iz}_{iy}", iy+1, iy+1, iz+1, iz+1)
            if proj_local.GetEntries() < Min_Entries:
                continue
            
            phi = iHist.GetYaxis().GetBinCenter(iy+1)
            phi_z.append(phi)
            phi_z_err.append(iHist.GetYaxis().GetBinWidth(iy+1)/2.)

            c1 = root.TCanvas("c1"+proj_local.GetName(),"c1"+proj_local.GetName(),720,360)
            gaus = root.TF1("gaus"+proj_local.GetName(), "gaus", -0.0025, 0.0025)
            proj_local.Draw()
            proj_local.Fit(gaus, "Q", "", -0.0025, 0.0025)
            c1.Update()
            if Do_save: c1.SaveAs("output/dynamic_delta_fits/" + proj_local.GetName() + ".png")

            Mean_z.append(gaus.GetParameter(1))
            Sigma_z.append(gaus.GetParameter(2))
            Mean_z_err.append(min(gaus.GetParError(1)*1000, gaus.GetParameter(2)/3))
            Sigma_z_err.append(min(gaus.GetParError(2), 0.05*gaus.GetParameter(2)))


            #int_fg_local.append(gaus.Integral(gaus.GetParameter(1)-sigma*gaus.GetParameter(2),gaus.GetParameter(1)+sigma*gaus.GetParameter(2)))
            #int_bg_local.append(2*gaus.Integral(gaus.GetParameter(1)+sigma*gaus.GetParameter(2),0.0025))

            #m2_pos.Draw("same") m2_neg.Draw("same")
            #c1.SaveAs( "output/dynamic_delta_fits/" + m2_name + ".png" )
        Mean_local.append(Mean_z)
        Mean_local_err.append(Mean_z_err)
        Sigma_local.append(Sigma_z)
        Sigma_local_err.append(Sigma_z_err)
        phi_local.append(phi_z)
        phi_local_err.append(phi_z_err)

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, phi_local, phi_local_err]


In [8]:
Ntr = N_layers
pool = mp.Pool(Ntr)
with pool:
    output_array = pool.map(sdphi_fitter, [i for i in range(Ntr)])
pool.close()

In [9]:
print(len(output_array[0][0]))

5


In [10]:
Means, Means_err, Sigmas, Sigmas_err, phis, phis_err= [], [], [], [], [], []
for i in range(N_layers):
    Means.append(output_array[i][0])
    Means_err.append(output_array[i][1])
    Sigmas.append(output_array[i][2])
    Sigmas_err.append(output_array[i][3])
    phis.append(output_array[i][4])
    phis_err.append(output_array[i][5])
print("Means", len(Means), len(Means[0]), len(Means[0][1]))

Means 4 5 14


In [11]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
fit_functions_0iter = []
for ifile in range(N_files):
    fit_functions0 = []
    for ilayer in range(N_layers):
        fit_functions1 = []
        for iphi in range(2):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_0iter_{ifile}_{iphi}_{ilayer}_{iOption0}","[0]+[1]*exp([2]*x)",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.0015)
            fit_functions1[-1].SetParameter(1,0.02)
            fit_functions1[-1].SetParameter(2,-3.3)
            fit_functions1[-1].SetParLimits(2,-8.126,-0.023)
            fit_functions1[-1].SetParLimits(0,0.001,0.002)
            if ilayer<2 : fit_functions1[-1].SetParLimits(2,-4.5,-2.5)
            elif ilayer<4 : fit_functions1[-1].SetParLimits(2,-4.0,-2.5)
            else : 
                fit_functions1[-1].SetParLimits(2,-6.0,-2.0)
                fit_functions1[-1].SetParLimits(1,0.0001,0.01)
            if iOption0%2==1: 
                fit_functions1[-1].SetParLimits(2,-5.126,-1.23)
                fit_functions1[-1].SetParameter(1,0.02)
                fit_functions1[-1].SetParLimits(1,0.001,0.6)
                fit_functions1[-1].SetParameter(0,0.005)
                fit_functions1[-1].SetParLimits(0,0.001,0.006)
            if is_sigmalised: 
                fit_functions1[-1].SetParLimits(0,0.,20)
                fit_functions1[-1].SetParLimits(1,0.,20)
                fit_functions1[-1].SetParLimits(2,-30,+10.23)
            fit_functions1[-1].SetLineColor(colors[iphi])
        fit_functions0.append(fit_functions1)
    fit_functions_0iter.append(fit_functions0)

In [12]:
g_sigma = []
h2=[]
c3 = root.TCanvas(f"c3",f"c3",1400,125*N_layers)
c3.Divide(2,int(N_layers/2),0,0)
Min, Max = 0., 0.003
names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
legends =[]
for ilayer in range(N_layers):
    c3.cd(ilayer+1)
    g_sigma_layer = []
    h2.append(Format_Pad_old( -1.5, 4.5, Min, Max, "p_{T} (GeV/#it{c})", "#sigma#"+iOptames[iOption0], 1.1, 1.1, 0.06, 0.06, "",0.15,0.15,0.,0.1))
    root.gPad.SetTopMargin(0.0)
    root.gPad.SetRightMargin(0.0)
    h2[-1].Draw()
    legends.append(root.TLegend(0.55,0.5,0.95,0.98,layers[ilayer]))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    for ized in range(len(Sigmas[ilayer])):
        n_phi = len(Sigmas[ilayer][ized])
        if n_phi<1: continue
        g_sigma_layer.append(root.TGraphErrors(n_phi, phis[ilayer][ized], Sigmas[ilayer][ized],phis_err[ilayer][ized], Sigmas_err[ilayer][ized]))
        Format_Graph(g_sigma_layer[-1],20,1, ized, 3, ized,1,1)
        g_sigma_layer[-1].Draw("same p")
    g_sigma.append(g_sigma_layer)
    legends[-1].Draw()
        
    
    #g_sigma_f.append(g_sigma_layer)
#g_sigma.append(g_sigma_f)
c3.Draw()
#c3.SaveAs(f"output/note/sigma_{ilayer}{iOption0}{is_sigmalised}.pdf")

In [13]:
fit_mean_functions = []
for ilayer in range(N_layers):
    fit_functions = []
    for ized in range(len(z_bins)):
        fit_functions.append(root.TF1(f"fit_functions_mean_iter_{ized}_"+layers[ilayer],"[0]/10000*sin(x)+[1]/10000*cos(x)+[2]",-1.5,4.5))
        fit_functions[-1].FixParameter(2,0.00)
        #fit_functions[-1].FixParameter(1,0.00)
    fit_mean_functions.append(fit_functions)

In [14]:
g_mean = []
h2=[]
c3 = root.TCanvas(f"c3",f"c3",1400,125*N_layers)
c3.Divide(2,int(N_layers/2),0,0)
Min, Max = -0.002, 0.002
names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
legends =[]
for ilayer in range(N_layers):
    c3.cd(ilayer+1)
    g_mean_layer = []
    h2.append(Format_Pad_old( -1.5, 4.5, Min, Max, "p_{T} (GeV/#it{c})", "#sigma#"+iOptames[iOption0], 1.1, 1.1, 0.06, 0.06, "",0.15,0.15,0.,0.1))
    root.gPad.SetTopMargin(0.0)
    root.gPad.SetRightMargin(0.0)
    h2[-1].Draw()
    legends.append(root.TLegend(0.55,0.5,0.95,0.98,layers[ilayer]))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    for ized in range(len(Means[ilayer])):
        n_phi = len(Means[ilayer][ized])
        if n_phi<1: continue
        g_sigma_layer.append(root.TGraphErrors(n_phi, phis[ilayer][ized], Means[ilayer][ized],phis_err[ilayer][ized], Means_err[ilayer][ized]))
        Format_Graph(g_sigma_layer[-1],20,1, ized, 3, ized,1,1)
        g_sigma_layer[-1].Fit(fit_mean_functions[ilayer][ized],"Q", "", -1.5, 4.5)
        g_sigma_layer[-1].Draw("same p")
    g_sigma.append(g_sigma_layer)
    legends[-1].Draw()
        
    
    #g_sigma_f.append(g_sigma_layer)
#g_sigma.append(g_sigma_f)
c3.Draw()
#c3.SaveAs(f"output/note/sigma_{ilayer}{iOption0}{is_sigmalised}.pdf")

In [15]:
for ilayer in range(N_layers):
    for ized in range(len(z_bins)):
        print(f"Layer {ilayer}, z_bin {ized}, dx: {fit_mean_functions[ilayer][ized].GetParameter(0):.6f} +/- {fit_mean_functions[ilayer][ized].GetParError(0):.6f}")
        print(f"Layer {ilayer}, z_bin {ized}, dy: {fit_mean_functions[ilayer][ized].GetParameter(1):.6f} +/- {fit_mean_functions[ilayer][ized].GetParError(1):.6f}")
        print(f"Layer {ilayer}, z_bin {ized}, c: {fit_mean_functions[ilayer][ized].GetParameter(2):.6f} +/- {fit_mean_functions[ilayer][ized].GetParError(2):.6f}")

Layer 0, z_bin 0, dx: 13.065978 +/- 7.792459
Layer 0, z_bin 0, dy: 13.924511 +/- 4.223575
Layer 0, z_bin 0, c: 0.000000 +/- 0.000000
Layer 0, z_bin 1, dx: 13.689802 +/- 3.018262
Layer 0, z_bin 1, dy: 9.417957 +/- 1.425985
Layer 0, z_bin 1, c: 0.000000 +/- 0.000000
Layer 0, z_bin 2, dx: -1.666510 +/- 3.924555
Layer 0, z_bin 2, dy: -0.496594 +/- 1.323303
Layer 0, z_bin 2, c: 0.000000 +/- 0.000000
Layer 0, z_bin 3, dx: -12.375107 +/- 3.042621
Layer 0, z_bin 3, dy: -2.986948 +/- 1.345850
Layer 0, z_bin 3, c: 0.000000 +/- 0.000000
Layer 0, z_bin 4, dx: -26.622301 +/- 3.238166
Layer 0, z_bin 4, dy: -5.263473 +/- 1.402445
Layer 0, z_bin 4, c: 0.000000 +/- 0.000000
Layer 1, z_bin 0, dx: -5.742482 +/- 5.705310
Layer 1, z_bin 0, dy: -6.328568 +/- 2.145501
Layer 1, z_bin 0, c: 0.000000 +/- 0.000000
Layer 1, z_bin 1, dx: 2.554210 +/- 3.645398
Layer 1, z_bin 1, dy: -9.651804 +/- 1.819658
Layer 1, z_bin 1, c: 0.000000 +/- 0.000000
Layer 1, z_bin 2, dx: 1.989251 +/- 2.811190
Layer 1, z_bin 2, dy: -4.

In [16]:
dynamic_dphi = []
dynamic_dphi_err = []
dphi = []
dphi_err = []

for ifile in range(0,1):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    dphi_ifile = []
    dphi_err_ifile = []
    for ilayer in range(N_layers):
        dynamic_dphi_ilayer = array( 'd' )
        dynamic_dphi_err_ilayer = array( 'd' )
        dphi_ilayer = array( 'd' )
        dphi_err_ilayer = array( 'd' )
        for iphi in range(N_phi):
            if iOption0==0 and not is_sigmalised:
                if abs( fit_mean_functions_0iter[ifile][ilayer][iphi].GetParameter(0) - ((0.002*phi_bins[2*iphi]-0.001-0.1))) > 0.0025: continue
                if ilayer>7 and (ilayer%4<2 and iphi<1) or (ilayer%4>1 and iphi>3) : continue
            elif iOption0==1 and not is_sigmalised:
                if (iphi<1 or iphi>4): continue
                if abs( fit_mean_functions_0iter[ifile][ilayer][iphi].GetParameter(0) - ((0.002*phi_bins[2*iphi]-0.001-0.1))) > 0.005: continue
            if not is_sigmalised and not new: dphi_ilayer.append( round(0.001*phi_bins[2*iphi]-0.0005-0.1,5))
            if new and not is_sigmalised: dphi_ilayer.append( round(0.002*phi_bins[2*iphi]-0.001-0.1,5))
            if is_sigmalised: 
                dphi_ilayer.append(round(0.2*phi_bins[2*iphi]-0.1-10,5))
            dphi_err_ilayer.append(0.001/4)
            dynamic_dphi_ilayer.append(fit_mean_functions_0iter[ifile][ilayer][iphi].GetParameter(0))
            dynamic_dphi_err_ilayer.append(max(0.0001,fit_mean_functions_0iter[ifile][ilayer][iphi].GetParError(0)))
        dynamic_dphi_ifile.append(dynamic_dphi_ilayer)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_ilayer)
        dphi_ifile.append(dphi_ilayer)
        dphi_err_ifile.append(dphi_err_ilayer)
    dynamic_dphi.append(dynamic_dphi_ifile)
    dynamic_dphi_err.append(dynamic_dphi_err_ifile)
    dphi.append(dphi_ifile)
    dphi_err.append(dphi_err_ifile)

print("dphi = ", dphi)
print(dynamic_dphi)

NameError: name 'N_phi' is not defined

In [ ]:
g_mean_phi = []
c4 = root.TCanvas(f"c40",f"c40",500,450)
h3=[]
legends =[]
for ifile in range(1):
    c4.cd(1+ifile)
    g_mean_phi_type=[]
    for itype in range(1):
        Min, Max, x0, x1 = -0.05, 0.05, -0.02, 0.05
        if is_sigmalised==1: Max, Min, x0, x1 = 1+itype*2, -1-itype*2, -2, 5
        names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
        for iphi in range(1):
            h3.append(Format_Pad_old( x0, x1, Min, Max, f"{iOptames[iOption0]}_prev [rad]", "#LT"+iOptames[iOption0]+"#GT [rad]", 1.1, 1.1, 0.06, 0.05, "",0.2,0.2,0.,0.1))
            h3[-1].Draw()
            legends.append(root.TLegend(0.55,0.4,0.85,0.9))
            legends[-1].SetFillColorAlpha(0,0)
            legends[-1].SetLineWidth(0)
            g_mean_phi_layer=[]
            for ilayer in range(0,N_layers):
                N_phi = len(dphi[ifile][ilayer])
                g_mean_phi_layer.append(root.TGraphErrors(N_phi, dphi[ifile][ilayer], dynamic_dphi[ifile][ilayer],dphi_err[ifile][ilayer],dynamic_dphi_err[ifile][ilayer]))
                g_mean_phi_layer[-1].SetName(f"test_graph_{itype}_{ifile}_{ilayer}_{iphi}_{iOption0}")

                Format_Graph(g_mean_phi_layer[-1],20+int(ilayer/4),1, colors[ilayer%4], 3, colors[ilayer%4],1,1)
                g_mean_phi_layer[-1].Fit("pol1","Q","")
                g_mean_phi_layer[-1].GetFunction("pol1").SetLineColor(colors[ilayer%4])
                g_mean_phi_layer[-1].Draw('sameP')
                legends[-1].AddEntry(g_mean_phi_layer[-1],layers[ilayer],"p")

            legends[-1].Draw()
        g_mean_phi_type.append(g_mean_phi_layer)
    g_mean_phi.append(g_mean_phi_type)
c4.Draw()
c4.SaveAs(f"output/Sigmas/mean_phi_prevphi_{ifile}{iOption0}{is_sigmalised}.png")


TypeError: none of the 9 overloaded methods succeeded. Full details:
  TGraphErrors::TGraphErrors(const TVectorT<float>& vx, const TVectorT<float>& vy, const TVectorT<float>& vex, const TVectorT<float>& vey) =>
    TypeError: takes at most 4 arguments (5 given)
  TGraphErrors::TGraphErrors(const TVectorT<double>& vx, const TVectorT<double>& vy, const TVectorT<double>& vex, const TVectorT<double>& vey) =>
    TypeError: takes at most 4 arguments (5 given)
  TGraphErrors::TGraphErrors(const TGraphErrors& gr) =>
    TypeError: takes at most 1 arguments (5 given)
  TGraphErrors::TGraphErrors(const TH1* h) =>
    TypeError: takes at most 1 arguments (5 given)
  TGraphErrors::TGraphErrors() =>
    TypeError: takes at most 0 arguments (5 given)
  TGraphErrors::TGraphErrors(int n) =>
    TypeError: takes at most 1 arguments (5 given)
  TGraphErrors::TGraphErrors(const char* filename, const char* format = "%lg %lg %lg %lg", const char* option = "") =>
    TypeError: takes at most 3 arguments (5

In [ ]:
if True:
    print("const float "+iOptames[iOption0]+f"_sigma_pt_params[{N_layers}][3]="+"\n{")
    for ilayer in range(len(fit_functions_0iter[0])):
        fits = fit_functions_0iter[0][ilayer]
        if iOption0%2==0:
            if ilayer < 8 :
                print("    {"+f"{round((fits[2].GetParameter(0)+fits[3].GetParameter(0))*0.5,6)}, {round((fits[2].GetParameter(1)+fits[3].GetParameter(1))*0.5,6)}, {round((fits[2].GetParameter(2)+fits[3].GetParameter(2))*0.5,6)}"+"},")
            elif ilayer%4<2:
                print("    {"+f"{round((fits[4].GetParameter(0)+fits[4].GetParameter(0))*0.5,6)}, {round((fits[4].GetParameter(1)+fits[4].GetParameter(1))*0.5,6)}, {round((fits[4].GetParameter(2)+fits[4].GetParameter(2))*0.5,6)}"+"},")
            else:
                print("    {"+f"{round((fits[0].GetParameter(0)+fits[1].GetParameter(0))*0.5,6)}, {round((fits[0].GetParameter(1)+fits[1].GetParameter(1))*0.5,6)}, {round((fits[0].GetParameter(2)+fits[1].GetParameter(2))*0.5,6)}"+"},")
        else:
            print("    {"+f"{round((fits[2].GetParameter(0)+fits[3].GetParameter(0))*0.5,6)}, {round((fits[2].GetParameter(1)+fits[3].GetParameter(1))*0.5,6)}, {round((fits[2].GetParameter(2)+fits[3].GetParameter(2))*0.5,6)}"+"},")
    print("};")
print("const float "+iOptames[iOption0]+f"_mean_"+iOptames[iOption0]+f"_params[{N_layers}][2]="+"\n{")
for ilayer in range(N_layers):
    print("    {"+f'{round(g_mean_phi[ifile][0][ilayer].GetFunction("pol1").GetParameter(0),6)}, {round(g_mean_phi[ifile][0][ilayer].GetFunction("pol1").GetParameter(1),6)}'+"},")
print("};")


const float phi_sigma_pt_params[16][3]=
{
    {0.00149, 0.002504, -2.730882},
    {0.00159, 0.004052, -2.897332},
    {0.001517, 0.002951, -2.922766},
    {0.001582, 0.003674, -3.030569},
    {0.001254, 0.003921, -2.315339},
    {0.001473, 0.004799, -2.618879},
    {0.001249, 0.004343, -2.631056},
    {0.001498, 0.003434, -2.144249},
    {0.001488, 0.004947, -2.001459},
    {0.001864, 0.009102, -3.286335},
    {0.001472, 0.006545, -2.364782},
    {0.001886, 0.005975, -2.327029},
    {0.001297, 0.005094, -3.729813},
    {0.001207, 0.003983, -3.321421},
    {0.001142, 0.005726, -3.623806},
    {0.001073, 0.002671, -2.094489},
};
const float phi_mean_phi_params[16][2]=
{
    {2.691981069501989e+38, 4.037978056699886e+39},
    {2.691981069501989e+38, 4.037978056699886e+39},
    {2.691981069501989e+38, 4.037978056699886e+39},
    {2.691981069501989e+38, 4.037978056699886e+39},
    {2.691981069501989e+38, 4.037978056699886e+39},
    {2.691981069501989e+38, 4.037978056699886e+39},
    {2.6919

In [ ]:
def sdphi_fitter(InHist, iphi, iFile=0, ilayer=0, sigma=2, fits_mean=0, fits_sigma=0, accur = 0.5):
    
    N_gaus=2
    seed0=iFile*ilayer*100
    iilayer = int(ilayer/arms)
    charge = (iilayer%2)*2 - 1
    factor1, factor2 = 1 + int(iilayer>0), 1 + 5*int(iilayer>0)
    hist_pt_bin = InHist.ProjectionZ("kek")

    pt_local =  array( 'd' )
    pt_local_err =  array( 'd' )

    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )
 
    for p in range(N_pt):
        c1 = root.TCanvas(f"c1{seed0+p}",f"c1{seed0+p}",720,360)

        m2_name=str(ilayer)+"/"+"F"+str(iFile)+"_P"+str(iphi)+"_dynamic_"+str(round(pt[p],1))

        var_porj = InHist.ProjectionX(f"sdphi{seed0+p}_{seed0+p}_{ilayer}_{iphi}",phi_bins[2*iphi],phi_bins[2*iphi],hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)
        var_porj.SetLineColor(1)

        if var_porj.GetEntries() < Min_Entries:
            continue

        pt_loc = round((pt_boarders[p]*2+pt_boarders[p+1])/3,2)
        pt_loc_err = (pt_boarders[p+1]-pt_boarders[p])/4.
        var_porj.SetTitle(f"dphi_{ilayer}_{iphi}={round(central_bean_prev,3)}_{pt_loc}")

        par_tot = array( 'd', (3*N_gaus+4)*[2.] )
        g_gaus= root.TF1( f'g_gaus_{seed0+p}_{ilayer}_{iphi}', 'gaus',  boards_fit[int(ilayer%2)][0]+central_bean_prev, boards_fit[int(ilayer%2)][1]+central_bean_prev)
        g_gaus2= root.TF1( f'g_gaus_2_{seed0+p}_{ilayer}_{iphi}', '[0]*TMath::Landau([3]*x,[1],[2])',  boards_fit2[iilayer][0]+central_bean_prev, boards_fit2[iilayer][1]+central_bean_prev)#[0]/[2]/sqrt(2*3.14159)*exp(-0.5*pow((x-[1])/[2],2))
        g_pol = root.TF1( f'g_pol{seed0+p}_{ilayer}_{iphi}', '[0]+[1]*x*(abs(x)/x-1)+[2]*x*(abs(x)/x+1)',  boards_tot[int(ilayer%2)][0],boards_tot[int(ilayer%2)][1])
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_pos{seed0+p}_{ilayer}_{iphi}', 'gaus(0)+[3]*TMath::Landau([9]*x,[4],[5])+[6]+[7]*x*(abs(x)/x-1)+[8]*x*(abs(x)/x+1)',boards_tot[int(ilayer%2)][0], boards_tot[int(ilayer%2)][1])
        Format_Func(total,2,6)
        g_gaus2.FixParameter(3,charge)

        if fits_mean == 0:
            g_gaus.SetParameter(1,0.)
            g_gaus.SetParameter(2,boards_fit[1][1])
            g_gaus.SetParLimits(1,boards_tot[int(ilayer%2)][0]/10+central_bean_prev,boards_tot[int(ilayer%2)][1]/10+central_bean_prev)
            g_gaus.SetParLimits(2,boards_fit[int(ilayer%2)][1]/10,2)
            g_gaus2.SetParameter(1,0.5*(boards_fit2[int(iilayer/2)][0]+boards_fit2[int(iilayer/2)][1])+central_bean_prev)
            g_gaus2.SetParameter(2,0.5*(boards_fit2[int(iilayer/2)][1]-boards_fit2[int(iilayer/2)][0]))
            g_gaus2.SetParLimits(0,1,10000000)
            g_gaus2.SetParLimits(1,boards_fit2[int(iilayer/2)][0]+central_bean_prev, boards_fit2[int(iilayer/2)][1]+central_bean_prev)
        else: 
            mean0 = fits_mean[0][ilayer][iphi].Eval(pt_loc)
            sigm0 = fits_sigma[0][ilayer][iphi].Eval(pt_loc)
            mean1 = mean0+charge*(0.005+0.01*math.exp(-3.33*pt_loc))*factor1
            sigm1 = sigm0*factor2
            g_gaus.SetParameter(1,mean0)
            g_gaus.SetParameter(2,sigm0)
            g_gaus2.SetParameter(1,mean1)
            g_gaus2.SetParameter(2,sigm1)

        var_porj.Draw()
        var_porj.Fit( g_gaus, 'QR')
        par_gaus = g_gaus.GetParameters()
        g_gaus2.SetParLimits(1, par_gaus[1]*charge+leftright[iilayer%2][0]*par_gaus[2], par_gaus[1]*charge+leftright[iilayer%2][1]*par_gaus[2] )
        g_gaus2.SetParLimits(2,leftright_sigma[iilayer%2][0]*par_gaus[2], leftright_sigma[iilayer%2][1]*par_gaus[2] )

        var_porj.Fit( g_gaus2, 'QR+' )
        var_porj.Fit( g_pol, 'QR+' )

        par_gaus2 = g_gaus2.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5] = par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2]
        par_tot[6], par_tot[7], par_tot[8], par_tot[9] = par_pol[0], par_pol[1], par_pol[1], charge
        total.FixParameter(9,charge)

        total.SetParameters( par_tot )
        total.SetParLimits(1,boards_tot[int(ilayer%2)][0]/10+central_bean_prev,boards_tot[int(ilayer%2)][1]/10+central_bean_prev)
        total.SetParLimits(2,boards_fit[int(ilayer%2)][1]/10,2)
        total.SetParLimits(3,par_gaus2[0]*0.49, par_gaus2[0]*2)
        total.SetParLimits(4,par_gaus2[1]-0.01, par_gaus2[1]+0.01)
        total.SetParLimits(5,par_gaus2[2]*0.49, par_gaus2[2]*8)
        #if iOption0==1:
        #    total.SetParameter(3,0.)
        #    total.SetParLimits(3,0,par_gaus2[0]*0.01)
            
        if not fits_mean==0: 
            mean0 = fits_mean[0][ilayer][iphi].Eval(pt_loc)
            sigm0 = fits_sigma[0][ilayer][iphi].Eval(pt_loc)
            mean1 = mean0+charge*(0.005+0.01*math.exp(-3.33*pt_loc))*factor1
            sigm1 = sigm0*factor2
            total.SetParameter(1,mean0)
            total.SetParameter(2,sigm0)
            total.SetParameter(1,mean1)
            total.SetParameter(2,sigm1)
        #total.SetParLimits(3,1,1000000)
        if iphi>-1:
            total.SetParLimits(1,boards_tot[int(ilayer%2)][0],boards_tot[int(ilayer%2)][1])
        #if iOption0%2==1: total.FixParameter(3,0.)
        
        var_porj.Draw()
        var_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()
        mean1 = par_tot[1]+charge*(0.005+0.01*math.exp(-3.33*pt_loc))*factor1
        sigm1 = par_tot[2]*factor2
        total.SetParameter(4, mean1)
        total.SetParLimits(4, par_tot[1]*charge+leftright[iilayer%2][0]*par_tot[2]/2, par_tot[1]*charge+leftright[iilayer%2][1]*par_tot[2])
        total.SetParLimits(5, par_tot[2]*0.5, 100*par_tot[2])
        
        var_porj.Draw()
        var_porj.Fit( total, 'QR' )
        par_tot = total.GetParameters()

        g_gaus= root.TF1( f'g_gaus_4_{seed0+p}_{ilayer}_{iphi}', 'gaus',  par_tot[1]-2*par_tot[2], par_tot[1]+2*par_tot[2])
        g_gaus2= root.TF1( f'g_gaus_5_{seed0+p}_{ilayer}_{iphi}', '[0]*TMath::Landau([3]*x,[1],[2])',  charge*par_tot[4]-10*par_tot[5], charge*par_tot[4]+10*par_tot[5])
        g_gaus2.FixParameter(3,charge)

        Format_Func(g_gaus,3,4)
        Format_Func(g_gaus2,3,root.kGreen+2)

        par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2] = par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5]
        par_pol[0], par_pol[1], par_pol[2] = par_tot[6], par_tot[7], par_tot[8]
        g_gaus2.SetParameters(par_gaus2)
        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_gaus2.Draw('same')
        g_pol.Draw('same')

        #if is_sigmalised:
        #    if abs(total.GetParameter(1))>1.5:
        #        continue
        if not is_sigmalised and iOption0==0 and pt[p]>1 and total.GetParameter(2)>0.003: continue

        
        pt_local.append(pt_loc)
        pt_local_err.append(pt_loc_err)

        Mean_local.append(total.GetParameter(1)-0*total.GetParameter(4)*charge)
        Sigma_local.append(total.GetParameter(2+3*0))
        if is_sigmalised==0:
            Mean_local_err.append (min(total.GetParError(1)+0.05*total.GetParameter(2),Sigma_local[-1]*0.5))
            Sigma_local_err.append(min(total.GetParError(2)+0.05*total.GetParameter(2),Sigma_local[-1]*0.3))
        else:
            Mean_local_err.append(min(total.GetParError(1)+ 0.1*1., 0.2))#(0.15-0.005*p)*total.GetParameter(2)
            Sigma_local_err.append(min(total.GetParError(2)+0.1*1., 0.1))#(0.15-0.005*p)*total.GetParameter(2)


        int_fg_local.append(g_gaus.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
        int_bg_local.append(g_pol.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
            
        #m2_pos.Draw("same") m2_neg.Draw("same")

        c1.SaveAs( "output/dynamic_delta_fits/" + m2_name + ".png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_bg_local, pt_local, pt_local_err]
